In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
retail_train=pd.read_csv(r'/kaggle/input/retail/Participants_Data_TGIH/Train.csv')
retail_test=pd.read_csv(r'/kaggle/input/retail/Participants_Data_TGIH/Test.csv')

In [ ]:
retail_train.head(5)

In [ ]:
retail_test.head(5)

In [ ]:
print(retail_train.shape,retail_test.shape)

In [ ]:
# Checking persentsge of null values in each column

In [ ]:
percent_missing = retail_train.isnull().sum() * 100 / len(retail_train)
print(percent_missing)

In [ ]:

percent_missing = retail_test.isnull().sum() * 100 / len(retail_test)
print(percent_missing)

In [ ]:
# Luckly we don't have any null values in the data set !!

In [ ]:
# Checking data type of each column 

In [ ]:
retail_train.dtypes

In [ ]:
retail_test.dtypes

In [ ]:
# We have 'InvoiceDate' as an object data type  

In [ ]:
retail_train['InvoiceDate'].head(5)

In [ ]:
retail_test['InvoiceDate'].head(5)

In [ ]:
# Since Time is not important for us, we need to keep the date 

In [ ]:
retail_train['InvoiceDate1']=pd.to_datetime(retail_train['InvoiceDate']).dt.date
retail_train['InvoiceTime']=pd.to_datetime(retail_train['InvoiceDate']).dt.time
retail_test['InvoiceDate1']=pd.to_datetime(retail_test['InvoiceDate']).dt.date
retail_test['InvoiceTime']=pd.to_datetime(retail_test['InvoiceDate']).dt.time

In [ ]:
retail_train.head(5)

In [ ]:
retail_test.head(5)

In [ ]:
retail_train.dtypes

In [ ]:

retail_test.dtypes

In [ ]:
# Now we need a function to split date_values

In [ ]:
def extract_date(data):
    date=data['InvoiceDate1']
    date_split=[]
    for day in date:
        date_split.append([day.year,day.month,day.day])
        #date_split.append([day.split('-')[0],day.split('-')[1],day.split('-')[2]])
        #date_split.append([day.split('-')[0],day.split('-')[1],'19'+day.split('-')[2] if int(day.split('-')[2])>=20 else '20'+day.split('-')[2]])
    new_date=pd.DataFrame(date_split,columns=['Year','Month','Date'])
    return new_date

In [ ]:
retail_train=pd.concat([retail_train,extract_date(retail_train)],axis=1)
retail_train=retail_train.drop(['InvoiceDate1'],axis=1)

In [ ]:
retail_test=pd.concat([retail_test,extract_date(retail_test)],axis=1)
retail_test=retail_test.drop(['InvoiceDate1'],axis=1)

In [ ]:
def extract_date(data):
    date=data['InvoiceTime']
    date_split=[]
    for day in date:
        date_split.append([day.hour,day.minute])
        #date_split.append([day.split('-')[0],day.split('-')[1],day.split('-')[2]])
        #date_split.append([day.split('-')[0],day.split('-')[1],'19'+day.split('-')[2] if int(day.split('-')[2])>=20 else '20'+day.split('-')[2]])
        
    new_date=pd.DataFrame(date_split,columns=['Hour','Minute'])
    
    return new_date

In [ ]:
retail_train=pd.concat([retail_train,extract_date(retail_train)],axis=1)
retail_train=retail_train.drop(['InvoiceTime'],axis=1)

In [ ]:
retail_test=pd.concat([retail_test,extract_date(retail_test)],axis=1)
retail_test=retail_test.drop(['InvoiceTime'],axis=1)

In [ ]:
retail_train['Time']=(retail_train['Hour']).astype(str)+'.'+(retail_train['Minute']).astype(str)
retail_train['Time']=pd.to_numeric(retail_train['Time'])
retail_train=retail_train.drop(['Hour','Minute'],axis=1)

In [ ]:
retail_test['Time']=(retail_test['Hour']).astype(str)+'.'+(retail_test['Minute']).astype(str)
retail_test['Time']=pd.to_numeric(retail_test['Time'])
retail_test=retail_test.drop(['Hour','Minute'],axis=1)

In [ ]:
retail_train.head(5)

In [ ]:
retail_test.head(5)

In [ ]:
 # Handning Cyclic data - 'InvoiceDate' - We need to convert these dates into useful data without loosing the cyclicity

In [ ]:
def code_cyclic_features(data,columns,drop_cols=True):
    for col in columns:
        max_val=max(data[col])

        data[col+'_sin']=np.sin(2*np.pi*data[col]/max_val)
        data[col+'_cos']=np.cos(2*np.pi*data[col]/max_val)

        if drop_cols:
            del data[col]
    return data  

In [ ]:
code_cyclic_features(retail_train,['Date','Month','Year','Time'])
code_cyclic_features(retail_test,['Date','Month','Year','Time'])

In [ ]:
# Now we need to check the Cyclicity of the data by ploating scatterplot

In [ ]:
import seaborn as sns

In [ ]:
sns.scatterplot(retail_train['Date_sin'],retail_train['Date_cos']).plot()

In [ ]:
sns.scatterplot(retail_test['Date_sin'],retail_test['Date_cos']).plot()

In [ ]:
sns.scatterplot(retail_train['Month_sin'],retail_train['Month_cos']).plot()

In [ ]:
sns.scatterplot(retail_test['Month_sin'],retail_test['Month_cos']).plot()

In [ ]:
sns.scatterplot(retail_train['Time_sin'],retail_train['Time_cos']).plot()

In [ ]:
sns.scatterplot(retail_test['Time_sin'],retail_test['Time_cos']).plot()

In [ ]:
sns.scatterplot(retail_train['Minute_sin'],retail_train['Minute_cos']).plot()

In [ ]:
sns.scatterplot(retail_test['Minute_sin'],retail_test['Minute_cos']).plot()

In [ ]:
retail_train.columns

In [ ]:
# now we can delete the columns which are not going to be useful for the prediction 

In [ ]:
retail_train=retail_train.drop(['InvoiceDate'],axis=1)
retail_test=retail_test.drop(['InvoiceDate'],axis=1)

In [ ]:
retail_train.head(5)

In [ ]:
retail_test.head(5)

In [ ]:
# Lets check the corrilation of the data

In [ ]:
sns.heatmap(retail_train.corr())

In [ ]:
# Handling Outlires

In [ ]:
dt=retail_train[['CustomerID','Country','Quantity','Description','StockCode','InvoiceNo']]
dt.boxplot()

In [ ]:
dt=retail_test[['CustomerID','Country','Quantity','Description','StockCode','InvoiceNo']]
dt.boxplot()

In [ ]:
retail_train.columns

In [ ]:
sns.boxplot(x=retail_train['CustomerID'])  

In [ ]:
sns.boxplot(x=retail_train['InvoiceNo'])  

In [ ]:
retail_train[retail_train.InvoiceNo<200].count()

In [ ]:
min_thresold, max_thresold = retail_train.InvoiceNo.quantile([0.001, 0.999])
min_thresold, max_thresold

#retail_train = retail_train[(retail_train.InvoiceNo<22000) & (retail_train.InvoiceNo>200)]
retail_train = retail_train[(retail_train.InvoiceNo<max_thresold) & (retail_train.InvoiceNo>min_thresold)]
retail_train.shape

In [ ]:
sns.boxplot(x=retail_train['StockCode'])  

In [ ]:
retail_train[retail_train.StockCode<50].count()

In [ ]:
min_thresold, max_thresold = retail_train.StockCode.quantile([0.001, 0.999])
min_thresold, max_thresold

#retail_train = retail_train[(retail_train.StockCode<3650) & (retail_train.StockCode>50)]
retail_train = retail_train[(retail_train.StockCode<max_thresold) & (retail_train.StockCode>min_thresold)]
retail_train.shape

In [ ]:
sns.boxplot(x=retail_train['Description'])  


In [ ]:
retail_train[retail_train.Description<50].count()

In [ ]:
min_thresold, max_thresold = retail_train.Description.quantile([0.001, 0.999])
min_thresold, max_thresold

#retail_train = retail_train[(retail_train.Description<3850) & (retail_train.Description>50)]
retail_train = retail_train[(retail_train.Description<max_thresold) & (retail_train.Description>min_thresold)]
retail_train.shape

In [ ]:
sns.boxplot(x=retail_train['Quantity'])  

In [ ]:
retail_train[retail_train.Quantity<-60000].count().sum()

In [ ]:
min_thresold, max_thresold = retail_train.Quantity.quantile([0.001, 0.999])
min_thresold, max_thresold

retail_train = retail_train[(retail_train.Quantity<60000) & (retail_train.Quantity>-60000)]
#retail_train = retail_train[(retail_train.Quantity<max_thresold) & (retail_train.Quantity>min_thresold)]
retail_train.shape

In [ ]:
sns.boxplot(x=retail_train['Country'])  

In [ ]:
retail_train['Country'].value_counts()

In [ ]:
retail_train[retail_train.Country<15].count().sum()

In [ ]:
min_thresold, max_thresold = retail_train.Country.quantile([0.001, 0.999])
min_thresold, max_thresold

#retail_train = retail_train[(retail_train.Quantity<60000) & (retail_train.Quantity>-60000)]
retail_train = retail_train[(retail_train.Country<max_thresold) & (retail_train.Country>min_thresold)]
retail_train.shape

In [ ]:
dt=retail_train[['Country','Quantity','Description','StockCode','InvoiceNo']]
dt.boxplot()

In [ ]:
dt=retail_test[['Country','Quantity','Description','StockCode','InvoiceNo']]
dt.boxplot()

In [ ]:
dt=retail_train[['Date_sin','Date_cos','Month_sin','Month_cos','Year_sin','Year_cos','Hour_sin','Hour_cos','Minute_sin','Minute_cos']]
dt.boxplot()

In [ ]:
dt=retail_test[['Date_sin','Date_cos','Month_sin','Month_cos','Year_sin','Year_cos','Hour_sin','Hour_cos','Minute_sin','Minute_cos']]
dt.boxplot()

In [ ]:
# Now our data set is ready - split the data as train and test for further analysis

In [ ]:
print(retail_train.shape,retail_test.shape)

In [ ]:
retail_train.reset_index(drop=True,inplace=True)
retail_test.reset_index(drop=True,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
r_train, r_test = train_test_split(retail_train, test_size = 0.2,random_state=2)

In [ ]:
r_train.columns

In [ ]:
x_train=r_train.drop(["UnitPrice"],1)
y_train=r_train["UnitPrice"]

x_test=r_test.drop(["UnitPrice"],1)
y_test=r_test["UnitPrice"]

In [ ]:
print(x_train.shape,x_test.shape)

In [ ]:
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)
x_test.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)
retail_test.reset_index(drop=True,inplace=True)

In [ ]:
# Now the data set is ready for - Feature Engineering

In [ ]:
X=x_train
y=y_train

In [ ]:
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor
import matplotlib.pyplot as plt

In [ ]:
model = RandomForestRegressor(n_jobs=-1)
model.fit(X,y)

In [ ]:
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(14).plot(kind='barh')
plt.show()

In [ ]:
imp_features=feat_importances.nlargest(11)
imp_features

In [ ]:
x_train=x_train.drop(["YEAR_cos",'YEAR_sin'],1)
x_test=x_test.drop(["YEAR_cos",'YEAR_sin'],1)
retail_test=retail_test.drop(["YEAR_cos",'YEAR_sin'],1)

In [ ]:
retail_test.columns

In [ ]:
x_train=x_train.drop(['YEAR_sin'],1)
x_test=x_test.drop(['YEAR_sin'],1)
retail_test=retail_test.drop(['YEAR_sin'],1)

In [ ]:
#Feature Selection- Dropping constant features

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
var_thres=VarianceThreshold(threshold=0)
var_thres.fit(x_train)

In [ ]:
var_thres.get_support()

In [ ]:
# non- constant features in the data set
sum(var_thres.get_support())

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = x_train.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.CMRmap_r)
plt.show()

In [ ]:

# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything other feature

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_features = correlation(x_train, 0.7) # setting corrilation threshold as 70 %
len(set(corr_features))

In [ ]:
corr_features

In [ ]:
x_train=x_train.drop(corr_features,axis=1)
x_test=x_test.drop(corr_features,axis=1)
retail_test=retail_test.drop(corr_features,axis=1)

In [ ]:
print(x_train.shape,x_test.shape)

In [ ]:
# Now we need to scale the data set

In [ ]:
x_train.describe()

In [ ]:
retail_test.describe()

In [ ]:
# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm_train = MinMaxScaler().fit(x_train)
norm_test = MinMaxScaler().fit(retail_test)

# transform training data
x_train_norm = norm_train.transform(x_train)
x_test_norm = norm_train.transform(x_test)

# transform testing dataabs
retail_test_norm = norm_test.transform(retail_test)

In [ ]:
x_train.columns

In [ ]:
# Now the data set is ready for Modeling  !!

In [ ]:
# LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm=LinearRegression(n_jobs=-1,copy_X=True,fit_intercept=True,normalize=False)

In [ ]:
lm.fit(x_train_norm,y_train)

In [ ]:
list(zip(x_train.columns,lm.coef_))

In [ ]:
lm.intercept_

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_test,lm.predict(x_test_norm))

In [ ]:
p=lm.predict(retail_test)

In [ ]:
submission=pd.DataFrame({
    
    'UnitPrice':p
})

In [ ]:
submission.to_csv('my_submission_file.csv', index=False)

In [ ]:
submission.head(5)

In [ ]:
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import KFold

In [ ]:
lamdas=np.linspace(1,100,100)
alphas=np.linspace(1,100,100)

In [ ]:
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)
x_test.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)
retail_test.reset_index(drop=True,inplace=True)

In [ ]:
mae_list=[]
for a in lamdas:
    ridge = Ridge(fit_intercept=True, alpha=a)
    
    kf = KFold(n_splits=10)
    
    xval_err = 0
    
    for train, test in kf.split(x_train):
        
        ridge.fit(x_train.loc[train], y_train[train])
        p = ridge.predict(x_train.loc[test])
        xval_err += mean_squared_error(y_train[test],p)
        
    mae_10cv = xval_err/10
    print(a, ':',mae_10cv)
    mae_list.extend([mae_10cv])

In [ ]:
best_alpha = lamdas[mae_list==min(mae_list)]
print('Alpha with min error:',best_alpha)

In [ ]:
# Expand alpha value 100

In [ ]:
lamdas=np.linspace(1,10,100)

In [ ]:
mae_list=[]
for a in lamdas:
    ridge = Ridge(fit_intercept=True, alpha=a)
    
    kf = KFold(n_splits=10)
    
    xval_err = 0
    
    for train, test in kf.split(x_train):
        
        ridge.fit(x_train.loc[train], y_train[train])
        p = ridge.predict(x_train.loc[test])
        xval_err += mean_squared_error(y_train[test],p)
        
    mae_10cv = xval_err/10
    print(a, ':',mae_10cv)
    mae_list.extend([mae_10cv])

In [ ]:
best_alpha = lamdas[mae_list==min(mae_list)]
print('Alpha with min error:',best_alpha)

In [ ]:
ridge = Ridge(fit_intercept=True, alpha=best_alpha)
ridge.fit(x_train,y_train)

In [ ]:
mean_squared_error(y_test,ridge.predict(x_test))

In [ ]:
# No improvements, trying Lasso

In [ ]:
lamdas=np.linspace(1,100,100)

In [ ]:
mae_list=[]
for a in lamdas:
    lasso = Lasso(fit_intercept=True, alpha=a)
    
    kf = KFold(n_splits=10)
    
    xval_err = 0
    
    for train, test in kf.split(x_train):
        
        lasso.fit(x_train.loc[train], y_train[train])
        p = lasso.predict(x_train.loc[test])
        xval_err += mean_squared_error(y_train[test],p)
        
    mae_10cv = xval_err/10
    print(a, ':',mae_10cv)
    mae_list.extend([mae_10cv])

In [ ]:
best_alpha = lamdas[mae_list==min(mae_list)]
print('Alpha with min error:',best_alpha)

In [ ]:
# Need to expand on 1

In [ ]:
lamdas=np.linspace(1,5,100)

In [ ]:
mae_list=[]
for a in lamdas:
    lasso = Lasso(fit_intercept=True, alpha=a)
    
    kf = KFold(n_splits=10)
    
    xval_err = 0
    
    for train, test in kf.split(x_train):
        
        lasso.fit(x_train.loc[train], y_train[train])
        p = lasso.predict(x_train.loc[test])
        xval_err += mean_squared_error(y_train[test],p)
        
    mae_10cv = xval_err/10
    print(a, ':',mae_10cv)
    mae_list.extend([mae_10cv])

In [ ]:
best_alpha = lamdas[mae_list==min(mae_list)]
print('Alpha with min error:',best_alpha)

In [ ]:
lasso = Lasso(fit_intercept=True, alpha=best_alpha)
lasso.fit(x_train,y_train)

In [ ]:
mean_squared_error(y_test,lasso.predict(x_test))

In [ ]:
# MAE is slightly improved

In [ ]:
p=lasso.predict(retail_test)

In [ ]:
submission=pd.DataFrame({
    
    'UnitPrice':p
})

In [ ]:
submission.to_csv('my_submission_file.csv', index=False)

In [ ]:
submission.head(5)

In [ ]:
# Lets try with DecisionTreeRegressor,RandomForestRegressor and ExtraTreesRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor 
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
dt = DecisionTreeRegressor()

In [ ]:
dt.fit(x_train,y_train)

In [ ]:
mean_squared_error(y_test,dt.predict(x_test))

In [ ]:
p=dt.predict(retail_test)

In [ ]:
submission=pd.DataFrame({
    
    'UnitPrice':p
})

In [ ]:
submission.to_csv('my_submission_file.csv', index=False)

In [ ]:
submission.head()

In [ ]:
rf=RandomForestRegressor(n_jobs=-1,verbose=1)

In [ ]:
rf.fit(x_train,y_train)

In [ ]:
mean_squared_error(y_test,rf.predict(x_test))

In [ ]:
et=ExtraTreesRegressor(n_jobs=-1,verbose=1)

In [ ]:
et.fit(x_train,y_train)

In [ ]:
mean_squared_error(y_test,et.predict(x_test))

In [ ]:
p=rf.predict(retail_test)

In [ ]:
submission=pd.DataFrame({
    
    'UnitPrice':p
})

In [ ]:
submission.to_csv('my_submission_file.csv', index=False)

In [ ]:
submission.head(5)

In [ ]:
# Trying with XGB  

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
'max_delta_step': 3, 'max_depth': 39, 'min_child_weight': 9, 'n_estimators': 3247, 'scale_pos_weight': 3   MSE : 176

In [ ]:
XGBModel = XGBRegressor(max_delta_step= 3, max_depth=39, min_child_weight= 9, n_estimators= 3247, scale_pos_weight= 3)
XGBModel.fit(x_train,y_train , verbose=False)

In [ ]:
# Get the mean squired error on the validation data :
XGBpredictions = XGBModel.predict(x_test)
MSE = mean_squared_error(y_test , XGBpredictions)
print('XGBoost validation MAE = ',MSE)

In [ ]:
136.96710552472163
163.23917464109846

In [ ]:
# XGB with CV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_dist={
               'max_depth':[i for i in range(1,50)],
                'min_child_weight':[i for i in range(1,10)],
                'max_delta_step':[i for i in range(1,10)],
                
                'scale_pos_weight':[i for i in range(1,50)],
                'n_estimators':[i for i in range(100,5000,100)]
   
            }

In [ ]:
#  max_delta_step=5, max_depth= 90, min_child_weight= 8, n_estimators= 2013, scale_pos_weight= 31

In [ ]:
XGB_clf=XGBRegressor()

In [ ]:
random_search=RandomizedSearchCV(XGB_clf,n_jobs=-1,verbose=1,cv=5,n_iter=10,#scoring='roc_auc',
                                 param_distributions=param_dist)

In [ ]:
random_search.fit (x_train,y_train)

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates=np.flatnonzero(results['rank_test_score']==i)
        for candidate in candidates:
            print('Model with rank: {0}'.format(i))
            print('Mean validation score: {0:3f}(std:{1:.5f})'.format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            print('parameters: {0}'.format(results['params'][candidate]))
            print('')

In [ ]:
XGB=random_search.best_estimator_

In [ ]:
report(random_search.cv_results_,5)

In [ ]:
XGBpredictions = XGB.predict(x_test)
MSE = mean_squared_error(y_test , XGBpredictions)
print('XGBoost validation MAE = ',MSE)

In [ ]:
# Hyper parameter tuning for XGB

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
def acc_model(params):
    XGB = XGBRegressor(**params)
    return cross_val_score(XGB, x_train, y_train,n_jobs=-1).mean()

In [ ]:
param_space = {
    'max_depth': hp.choice('max_depth', range(1,100)),
    
    'n_estimators': hp.choice('n_estimators', range(100,5000)),
    
    
    'min_child_weight': hp.choice('min_child_weight',range(1,10)),
    'max_delta_step': hp.choice('max_delta_step',range(1,10)),
    'scale_pos_weight':hp.choice('scale_pos_weight',range(1,50))

}

In [ ]:
best = 0
def f(params):
    global best
    acc = acc_model(params)
    if acc > best:
        best = acc
    print ('new best:', best, params)
    return {'loss': -acc, 'status': STATUS_OK}

In [ ]:
trials = Trials()
best = fmin(f, param_space, algo=tpe.suggest, max_evals=10, trials=trials)
print ('best:')
print (best)

In [ ]:
p=XGBModel.predict(retail_test)

In [ ]:
submission=pd.DataFrame({
    
    'UnitPrice':p
})

In [ ]:
submission.to_csv('my_submission_file.csv', index=False)

In [ ]:
submission.head(5)

In [ ]:
# CNN

In [ ]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = x_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

In [ ]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
NN_model.fit(x_train, y_train, epochs=500, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

In [ ]:
Weights-497--1.35965.hdf5

In [ ]:
wights_file = 'Weights-497--1.35965.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [ ]:
predictions = NN_model.predict(x_test)

In [ ]:
MSE = mean_squared_error(y_test , predictions)
print('XGBoost validation MAE = ',MSE)

In [ ]:
p=NN_model.predict(retail_test)

In [ ]:
submission=pd.DataFrame({
    
    'UnitPrice':p[:,0]
})

In [ ]:
submission.to_csv('my_submission_file.csv', index=False)

In [ ]:
submission.head(5)

In [ ]:
from timeit import default_timer as timer
from sklearn import linear_model
# store data as an array
X = np.array(x_train)
# again, timing the function for comparison
start_kfold = timer()
# use cross_val_predict to generate K-Fold predictions
lm_k = linear_model.LinearRegression()
k_predictions = cross_val_predict(lm_k, X, y_train, cv=10)
print(k_predictions)
end_kfold = timer()
kfold_time = (end_kfold - start_kfold)